This file includes a simple use case of the module.

A timed automata system is read from an xml file, an edge constraint is changed and the system is written back.

In [ ]:
from libta import *

system = UTAP.TimedAutomataSystem() # Create a system
parseXMLFile("examples/literature/CAS.xml", system, True) # Read system from file, 0 indicates success.

In [ ]:
templates = system.getTemplates() # Getting templates of the system

states_with_invariants = []

for template in templates:
        for state in template.states:
                if state.invariant.getSize() > 0: # Access states which have an invariant
                        states_with_invariants.append(state)
                        print("State uid:", state.uid) # Get state name
                        print("Invariant:", state.invariant) # Get state invariant
                        # print(state.toString()) # Print state in a readable form

In [ ]:
# Getting the edge guards of the system
Constants = UTAP.Constants

edge_guards_with_clock = []
for template in templates:
    for edge in template.edges:
        if (edge.guard.usesClock()): # Add guards that uses clocks to a list
            print("Source:", edge.src.uid, "; Dest:", edge.dst.uid)
            if edge.sync.getSize() > 0:
                print("Edge sync:", edge.sync)

            if edge.assign.getSize() > 0:
                print("Edge assign:", edge.assign)
                
            edge_guards_with_clock.append(edge.guard)

print(edge_guards_with_clock[0])
if edge_guards_with_clock[0].getKind() == Constants.AND:
    print("Guard has kind AND")

# Accessing subexpressions of expressions
print("Guard has", edge_guards_with_clock[0].getSize(), "subexpressions")
for i in range(edge_guards_with_clock[0].getSize()):
    print("Subexpression", i, edge_guards_with_clock[0][i])

In [ ]:
# Changing expression values
position = edge_guards_with_clock[0][0][1].getPosition() # Position of the expression
t = edge_guards_with_clock[0][0][1] # Sub-subexpression of the guard expression
if t.getKind() == Constants.CONSTANT:
    print("Const. old t=" + str(t.getValue()))
    #print("Double old t=" + str(t.getDoubleValue()))

exp = t.createConstant(100, position)
#exp = t.createDouble(100, position) # Creating double type expression
edge_guards_with_clock[0][0][1] = exp
if t.getKind() == Constants.CONSTANT:
    print("Const. new t=" + str(t.getValue()))
    #print("Double new t=" + str(t.getDoubleValue()))

In [ ]:
writeXMLFile("example.xml", system) # Write the system back to a file, 0 indicates success.

Cells above demonstrated the basic use of the wrapper, using the functions and classes generated directly from UTAP library. For further usage please refer to its documentation. From now on, usage of some helper classes and functions that this library has for itself will be shown.

In [1]:
# Changing expression values with helper classess and functions
from libta import *
from libta.nta import *

nta = NTAHelper("examples/generator/test5_6_2.xml", "nta") # Get the system with helper class

edge_guards_with_clock = []
for template in nta.templates:
	for edge in template.edges:
		if edge.guard.usesClock():
			edge_guards_with_clock.append(edge.guard)

exp_list = get_expression_list(edge_guards_with_clock[4])
print(exp_list[0].toString())
print(edge_guards_with_clock[4].toString())

x4 <= 14
x4 <= 14 && x0 >= 11 && x2 <= 15


In [2]:
change_expression_value(exp_list[0], 100)
print(exp_list[0].toString())
print(edge_guards_with_clock[4].toString()) # Changes in the expression objects are reflected to all its instances except its deep clones

x4 <= 100
x4 <= 100 && x0 >= 11 && x2 <= 15


In [3]:
from libta.path_analysis import *

# Construct a path from string labels of the states
# The resulting path is a list of the edges between states, with its first element being the initial state of the path.
# If no such edge is found, an empty list is returned.
path = ["l2", "l3", "l4", "l5", "l6", "l7", "l1"]
path = construct_path_from_labels(path, nta.templates[0])
print(path)

path_wrong = ["l2", "l3", "l4", "l5", "l6", "l1"]
path_wrong = construct_path_from_labels(path, nta.templates[0])
print(path_wrong)

[<cppyy.gbl.UTAP.state_t object at 0x563e0e726c20>, <cppyy.gbl.UTAP.edge_t object at 0x563e0d9f2c58>, <cppyy.gbl.UTAP.edge_t object at 0x563e0d9f2cf0>, <cppyy.gbl.UTAP.edge_t object at 0x563e0d9ed7a0>, <cppyy.gbl.UTAP.edge_t object at 0x563e0d9ed838>, <cppyy.gbl.UTAP.edge_t object at 0x563e0d9ed8d0>, <cppyy.gbl.UTAP.edge_t object at 0x563e0d7ac0a0>]
[]


In [4]:
for edge in path[1:]:
        # State objects, (e.g. edge.dst) has unique uids, and they can be compared or printed directly
        # symbol.getName() can be used to get the string label of the symbol, in this case the state uid
        print("From:", edge.src.uid.getName(), "\tto:", edge.dst.uid)

print(path_exists(path))
print(find_used_clocks(path[1:])) # find_used_clocks takes the edge list as its argument

From: l2 	to: (location) l3
From: l3 	to: (location) l4
From: l4 	to: (location) l5
From: l5 	to: (location) l6
From: l6 	to: (location) l7
From: l7 	to: (location) l1
True
{b'x3', b'x0', b'x1', b'x2', b'x4'}


In [5]:
nta = NTAHelper("tests/testcases/path_realizable/test26.xml", "nta1")
path = construct_path_from_labels(["l0", "l1", "l2"], nta.templates[0]) # Construct a path from string labels of the states
# Check if the path is realizable, with the options of no initial clock valuations, 
# and printing the constraint matrices and created LP problem
is_path_realizable(path, None, True, True)

[1, 0] <= 5
[1, 0] <= 3
[-1, -1] <= -100
\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 3
\   Variables        : 2
\     Binary         : 0
\     Integer        : 2
\     Continuous     : 0
Minimize
 Obj: 
Subject to
 auto_c_000000000: +1 x[0]  <= 5
 auto_c_000000001: +1 x[0]  <= 3
 auto_c_000000002: -1 x[0] -1 x[1]  <= -100
Bounds
 0 <= x[0] <= inf
 0 <= x[1] <= inf
Generals
 x[0]
 x[1]
End



(True, [3.0, 97.0])

In [ ]:
nta.writeToXML()